In [ ]:
# Только при необходимости в Google Colab или локально, если нет нужных пакетов:
%pip install sentence-transformers scikit-learn pandas numpy
# Если планируется fine-tuning: 
%pip install torch

1. Загрузка большого датасета репозиториев

In [2]:
import pandas as pd
import numpy as np

# Предположим, что у нас есть CSV-файл "big_repos.csv",
# который содержит столбцы: repoName, description
big_repos_df = pd.read_csv(r"C:\Users\sharp\Desktop\nuget-master\src\nuget-master\src\python-scripts\dataset\preprocessed_txtdata.csv")
# Убедимся, что колонки действительно существуют:
print("Big repos head:")
print(big_repos_df.head())


Big repos head:
                          Repository  \
0                  leiurayer/downkyi   
1  BluePointLilac/ContextMenuManager   
2            microsoft/reverse-proxy   
3         Kyome22/RunCat_for_windows   
4                         dotnet/tye   

                                         Description  
0                                       downkyi8khdr  
1                                            windows  
2  toolkit developing highperformance http revers...  
3         cute running cat animation windows taskbar  
4  tye tool makes developing testing deploying mi...  


2. Загрузка небольшого ручного датасета

In [3]:
# Это набор [query, repoName, repo_description, label].
# label = 1 (релевантно), 0 (нерелевантно).
# Если нет repo_description в самом файле, 
#   можно подтягивать из big_repos_df. Но пусть будет сразу:

labeled_df = pd.read_csv(r"C:\Users\sharp\Desktop\nuget-master\src\nuget-master\src\python-scripts\dataset\test_dataset_with_descriptions.csv")
print("Labeled dataset head:")
print(labeled_df.head())


Labeled dataset head:
                          query  \
0                      jwt auth   
1                      jwt auth   
2       Kafka messages consumer   
3       Kafka messages consumer   
4  Weather forecast application   

                                            repoName  \
0   mostakahammed/JwtAuthenticationWithEFCoreIdenity   
1                           Andrej-Gorlov/ProductAPI   
2                confluentinc/confluent-kafka-dotnet   
3                           ExactTargetDev/kafka-net   
4  Nucleus-center-of-excellence/weatherman-dotnet...   

                                    repo_description  label  
0  Implement JWT Authentication & Role base Autho...      1  
1                                        Product API      1  
2               Confluent's Apache Kafka .NET client      1  
3  This is a .NET implementation of a client for ...      1  
4  The idea of the application is to design a Wea...      1  


3. Подключение предобученной модели Sentence-BERT

In [4]:
from sentence_transformers import SentenceTransformer

# Загрузка конкретной модели
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# Или 'multi-qa-MiniLM-L6-cos-v1', 'all-mpnet-base-v2', 'all-distilroberta-v1' и т.д.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

4. Генерация эмбеддингов для основного набора репозиториев

In [ ]:
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

# 4.1 Извлекаем описания
all_repo_texts = big_repos_df['Description'].fillna("").tolist()

# 4.2 Векторизуем (batch-encode)
print("Encoding main repo descriptions...")
repo_embeddings = model.encode(all_repo_texts, 
                               batch_size=32, 
                               show_progress_bar=True, 
                               convert_to_numpy=True)

# 4.3 Строим kNN индекс (по косинусному сходству)
knn = NearestNeighbors(metric='cosine', n_neighbors=5)
knn.fit(repo_embeddings)


In [7]:
# Подготовим быстрый доступ к repoName: 
# (чтобы при поиске по индексу знать, какой репозиторий вернули)
repo_names = big_repos_df['Repository'].tolist()  # список по индексам 0..N-1

5. Функция «поиска» для произвольного запроса

In [8]:
def search_top_k(query_text, k=5):
    """
    Возвращает список словарей вида 
    [
      {repoIndx: i, repoName: "..."},
      {repoIndx: j, repoName: "..."},
      ...
    ]
    наиболее релевантных к запросу (по косинусной близости).
    """
    query_emb = model.encode([query_text], convert_to_numpy=True)
    distances, indices = knn.kneighbors(query_emb, n_neighbors=k)

    # indices[0] - это индексы в big_repos_df
    results = []
    for rank, idx in enumerate(indices[0]):
        repo_info = {
            'repoIndx': idx,
            'repoName': repo_names[idx]
        }
        results.append(repo_info)
    return results

# Тест
query = "jwt auth"
results = search_top_k(query, k=5)
print(f"Query: '{query}' -> top 5 repos:\n", results)


Query: 'jwt auth' -> top 5 repos:
 [{'repoIndx': 4570, 'repoName': 'cornflourblue/dotnet-6-jwt-authentication-api'}, {'repoIndx': 1612, 'repoName': 'cornflourblue/dotnet-6-jwt-authentication-api'}, {'repoIndx': 670, 'repoName': 'cornflourblue/dotnet-6-jwt-authentication-api'}, {'repoIndx': 6007, 'repoName': 'cornflourblue/dotnet-6-jwt-authentication-api'}, {'repoIndx': 3053, 'repoName': 'cornflourblue/dotnet-6-jwt-authentication-api'}]


6. Работа с ручным датасетом (train/val/test) для теста и/или fine-tuning

6.1 Разбиваем labeled_df
Если у нас 30–40 примеров, делаем:

* 20% (test) — в сторону, для финальной проверки
* 80% (train+val) — для обучения и/или подбора параметров

In [9]:
from sklearn.model_selection import train_test_split

if len(labeled_df) > 0:
    trainval_df, test_df = train_test_split(labeled_df, test_size=0.2, random_state=42)
    print(f"Train+Val: {len(trainval_df)}, Test: {len(test_df)}")
else:
    trainval_df, test_df = None, None


Train+Val: 33, Test: 9


6.2 Если мы не дообучаем модель

In [10]:
def compute_precision_recall_at_k(df_test, k=5):
    """
    df_test: DataFrame c колонками [query, repoName, label].
    Для каждой строки с label=1 проверяем, попадает ли repoName в top-k результатов search_top_k(query).
    Возвращаем общий precision, recall по всем query.
    """
    total_predictions = 0         # sum(K) по всем запросам
    total_relevant = 0           # сколько всего label=1
    total_correct = 0            # скольких релевантных мы верно поймали

    queries = df_test['query'].unique()
    for q in queries:
        subset = df_test[df_test['query'] == q]
        # Сколько релевантных (label=1) для этого q
        relevant_subset = subset[subset['label'] == 1]
        num_relevant = len(relevant_subset)
        total_relevant += num_relevant

        # Выполним поиск top-k
        top_results = search_top_k(q, k=k)
        top_repo_names = {r['repoName'] for r in top_results}

        # Подсчитываем, сколько из релевантных попало в top-k
        correct_for_q = 0
        for idx, row in relevant_subset.iterrows():
            if row['repoName'] in top_repo_names:
                correct_for_q += 1

        total_correct += correct_for_q
        total_predictions += k  # для всех запросов выдаём k репозиториев

    precision = total_correct / total_predictions if total_predictions > 0 else 0.0
    recall = total_correct / total_relevant if total_relevant > 0 else 0.0
    return precision, recall

# Если есть test_df, считаем
if test_df is not None and len(test_df) > 0:
    p5, r5 = compute_precision_recall_at_k(test_df, k=5)
    print(f"Baseline (no fine-tuning) precision@5={p5:.3f}, recall@5={r5:.3f}")


Baseline (no fine-tuning) P@5=0.025, R@5=0.111


# Сохранимс модели

In [30]:
import joblib

In [31]:
model.save('model/trained_models/sbert_model_folder')
joblib.dump(knn, 'knn_model.joblib')

['knn_model.joblib']

6.3 (Опционально Если хотим дообучить) Fine-tuning (Siamese/Bi-Encoder) на trainval_df

# 7. Итоговое использование
* model (Sentence-BERT, возможно не дообученная),
* repo_embeddings — векторное представление каждого репозитория из big_repos_df,
* knn — построенный индекс для быстрого поиска top-K.

In [4]:
input_query = "kafka adapter"
top_results = search_top_k(input_query, k=5)

for rep in top_results: print(rep['repoName'])
# print(top_results)
# [{repoIndx:..., repoName:...}, ...]


NameError: name 'search_top_k' is not defined

In [ ]:
%pip install PyGithub
import sys
import re
from github import Github, GithubException 
from github import Auth


def get_packages_from(repository: str):

    # Get the repository object
    repo = g.get_repo(repository)

    # Get all branches in the repository
    branches = repo.get_branches()

    # Get the default branch (usually the root branch)
    default_branch = next((branch for branch in branches if branch.name == repo.default_branch), None)

    # Get the root tree of the repository
    tree = repo.get_git_tree(sha=default_branch.commit.sha, recursive=True)

    # Filter out only the.csproj files
    csproj_files = []
    for item in tree.tree:
        if item.type == "blob" and item.path.endswith(".csproj"):
            csproj_files.append(item)

    # Extract PackageReference from each.csproj file
    package_references = set()
    for file in csproj_files:
        file_content = repo.get_contents(file.path).decoded_content.decode("utf-8")
        pattern = r'PackageReference Include="([^"]+)"'
        matches = re.findall(pattern, file_content)
        package_references.update(matches)

    return package_references